In [2]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 'sports-classification'

mkdir: /Users/rondineli/.kaggle: File exists
cp: kaggle.json: No such file or directory
chmod: /Users/rondineli/.kaggle/kaggle.json: No such file or directory
Traceback (most recent call last):
  File "/Users/rondineli/miniconda3/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/Users/rondineli/miniconda3/lib/python3.12/site-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/Users/rondineli/miniconda3/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /Users/rondineli/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [1]:
from typing import Optional

import tensorflow as tf
from tensorflow import keras
from keras.models import Model, Sequential
from keras.layers import Activation, Conv2D, Permute, multiply, Add, ReLU, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, GlobalAveragePooling2D, Reshape, Multiply
from keras import Input
from typing import Optional
from tensorflow.keras.regularizers import l2
from keras import backend as K


      


In [129]:
class SeResNet():
    def __init__(self, num_classes: Optional[int] = 100):
        self.shape = (32, 32, 3)
        self.ratio = 16
        self.default_activation = 'relu'
        self.inputs = Input(shape=self.shape)
        self.num_classes = num_classes
    
    @property
    def name(self):
        return f'se_res_net_{self.num_classes}'

    def main(self):
        ''' controller main to call from class model '''
        return keras.Model(self.inputs, self.se_res_net())

    def squeeze_excite_net(self, input_tensor) -> keras.KerasTensor:
        ''' Squeeze-and-Excitation (SE) Block '''
        channel_axis = -1 if K.image_data_format() == "channels_last" else 1
        filters = int(input_tensor.shape[channel_axis])

        se_net = GlobalAveragePooling2D()(input_tensor)
        se_net = Dense(filters // self.ratio, activation=self.default_activation, kernel_initializer='he_normal', use_bias=False)(se_net)
        se_net = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se_net)

        se_net = Reshape((1, 1, filters))(se_net)
        return Multiply()([input_tensor, se_net])

    def residual_block(self, input_tensor, filters: int) -> keras.KerasTensor:
        ''' Residual Block with Channel Matching '''
        res_block = Conv2D(filters, (3, 3), padding='same', activation=self.default_activation)(input_tensor)
        res_block = BatchNormalization()(res_block)
        res_block = ReLU()(res_block)

        res_block = Conv2D(filters, (3, 3), padding='same')(res_block)
        res_block = BatchNormalization()(res_block)

        shortcut = Conv2D(filters, (1, 1), padding='same')(input_tensor)
        res_block = Add()([res_block, shortcut])
        return ReLU()(res_block)

    def se_res_net(self, filters: Optional[int] = 128):
        ''' SE-ResNet Architecture '''
        self.tensor = Conv2D(32, (3, 3), activation=self.default_activation, padding='same')(self.inputs)
        self.tensor = BatchNormalization()(self.tensor)
        self.tensor = Activation('relu')(self.tensor)

        self.tensor = self.residual_block(self.tensor, 64)
        self.tensor = self.squeeze_excite_net(self.tensor)

        self.tensor = Conv2D(filters, (3, 3), padding="same", kernel_regularizer=l2(0.001), activation=self.default_activation)(self.tensor)
        self.tensor = BatchNormalization()(self.tensor)
        self.tensor = Dropout(0.7)(self.tensor)
        self.tensor = GlobalAveragePooling2D()(self.tensor)
        self.tensor = Dropout(0.7)(self.tensor)
        return Dense(self.num_classes, activation='softmax')(self.tensor)

    def __repr__(self):
        return f'SeResNet <{self.num_classes}>'





In [120]:
model = SeResNet().main()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
